## 3. Transformer Architecture and Pre-Trained Models

### Theoretical Concepts

#### Transformer Architecture

The Transformer architecture, introduced by Vaswani et al. in the paper ["Attention Is All You Need" (2017)](https://arxiv.org/abs/1706.03762), revolutionized the field of NLP. Unlike recurrent neural networks (RNNs) or convolutional neural networks (CNNs) commonly used in sequence-to-sequence tasks, Transformers rely solely on **attention mechanisms** to handle dependencies between tokens, regardless of their positions in the input sequence.

Key components:

1. **Attention Mechanism**:  
   The core idea is to allow the model to focus on different parts of the input sequence when computing representations. Instead of processing inputs strictly in order, attention allows each token to "look at" other tokens and weigh their importance for building its representation.

2. **Self-Attention**:  
   Within the Transformer, each token in a sequence attends to all other tokens, producing a rich context-aware embedding for each position. This allows for parallelization and capturing long-range dependencies more efficiently than RNNs.

3. **Encoder-Decoder Structure**:  
   A standard Transformer consists of an **Encoder** and a **Decoder**:
   - The **Encoder** reads the input sequence and produces a contextualized representation.
   - The **Decoder** uses the encoder’s output and previously generated tokens to produce the next token in the output sequence.
   
   For models like BERT, which focus on understanding and encoding text, only the encoder portion is used. For models like GPT-2, which focus on text generation, only the decoder (or a decoder-like) portion is utilized.

4. **Positional Embeddings**:  
   Since Transformers do not process sequences in order (no inherent recurrence), they use positional embeddings to provide information about the position of each token in the sequence.

#### Pre-Trained Models (BERT, GPT-2, DistilGPT-2)

1. **BERT (Bidirectional Encoder Representations from Transformers)**:  
   BERT is an encoder-only Transformer model trained on a large corpus of text with a masked language modeling objective. It learns deep bidirectional representations, making it excellent for understanding tasks (e.g., sentiment analysis, question answering).

2. **GPT-2 (Generative Pre-trained Transformer 2)**:  
   GPT-2 is a decoder-only Transformer model trained as a language model to predict the next token. It generates coherent and contextually relevant text, making it well-suited for tasks like text generation, summarization, and conversation modeling.

3. **DistilGPT-2**:  
   DistilGPT-2 is a distilled (compressed) version of GPT-2. Distillation retains much of the capabilities of GPT-2 but in a lighter, smaller model, making it faster and more efficient to run locally, even without a GPU.

#### Differences Among the Models

- **Encoder-Only (BERT)**: Good for understanding the meaning of input text; tasks often involve classification or extracting meaningful features from text.
- **Decoder-Only (GPT-2, DistilGPT-2)**: Good for generating text, including completing prompts and writing coherent paragraphs.
- **Size and Efficiency**:  
  DistilGPT-2 is smaller and faster than GPT-2, making it more practical for local inference on CPU.

## Parameters: Temperature, do_sample, top_k, and top_p

When a language model (like GPT-2 or DistilGPT-2) generates text, it doesn't just pick the most likely next word every time. Instead, it often samples from a probability distribution over possible next words. This randomness can make the text more creative, less repetitive, and more human-like. The parameters `temperature`, `do_sample`, `top_k`, and `top_p` help control this randomness and the "creativity" of the generated text.

- **Temperature**:  
  This parameter adjusts the "confidence" of the model when choosing the next token.  
  - A **high temperature** (>1) makes the model more random: it's more likely to choose less probable words, leading to more variety and sometimes more chaotic or surprising output.  
  - A **low temperature** (<1) makes the model more conservative: it sticks more closely to the top predicted words. A temperature of 1 is often considered the baseline.  
   
  Think of temperature like a "spice level" in cooking. If you set it high, you add more unpredictable flavors. If you set it low, you keep things mild and predictable.

- **do_sample**:  
  If you set `do_sample=True`, the model will randomly sample from the probability distribution of possible next tokens. If `do_sample=False`, the model will always pick the token with the highest probability.  
   
  Sampling is what introduces variability into the generated text. If you never sample and always pick the top token (the model’s "best guess"), you might get repetitive or boring text.

- **top_k**:  
  This setting limits the number of candidates the model can choose from for the next token. If `top_k=50`, for example, the model will only consider the top 50 most likely next words and ignore the rest. This truncation can help avoid bizarre or extremely rare words.  
   
  By setting a smaller `top_k`, you ensure the model focuses on a more likely subset of words. This often improves coherence but can reduce creativity.

- **top_p** (also known as nucleus sampling):  
  Instead of picking the next word from a fixed number of top words (`top_k`), `top_p` considers the cumulative probability. For instance, `top_p=0.9` means the model will choose from the smallest set of words whose total probability sums up to 90%. If the first two words together have a 75% probability and adding the third word gets you to 92%, then `top_p=0.9` would consider just those first two words.  
   
  This approach dynamically adjusts the number of candidates based on their collective probability. The result is often more balanced creativity than a fixed `top_k`.

**In summary:**

- **temperature**: Controls how "adventurous" the model is when picking next words.
- **do_sample**: Decides whether to pick the single best next word or randomly sample among possible words.
- **top_k**: Limits the pool of next-word candidates to the top K words.
- **top_p**: Limits the pool of next-word candidates to those accounting for a certain cumulative probability.

By tweaking these parameters, you can make the generated text more predictable or more imaginative, depending on what you need.

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load a lightweight pre-trained model and tokenizer (DistilGPT-2)
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Ensure we're using CPU
device = torch.device("cpu")
model.to(device)

# Prompt text
prompt = "In a distant future, artificial intelligence"

# Encode the input prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

# Generate text
# max_length: the total length including prompt
# do_sample: enable sampling for more creative text
# top_k, top_p: parameters for sampling
output_ids = model.generate(
    input_ids,
    max_length=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)

# Decode generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a distant future, artificial intelligence will be able to control the movements of the entire species in order to control the movement of species. However, the technology is limited to humans, so we will need to learn more about the technology.
